# Fitting a simple slab model

In order to show one of the simplest analyses that `EasyReflectometry` can perform, we will use the [great example from the *refnx* documentation](https://refnx.readthedocs.io/en/latest/getting_started.html#Fitting-a-neutron-reflectometry-dataset).
This involves the analysis of a single neutron reflectometry dataset from a hydrated polymer film system. 
Before we start on any analysis, we will import the necessary packages and functions.

In [ ]:
import EasyReflectometry
import refnx
from EasyReflectometry.data import load
from EasyReflectometry.sample import Layer, Structure
from EasyReflectometry.sample.material import Material
from EasyReflectometry.sample.item import MultiLayer
from EasyReflectometry.experiment.model import Model
from EasyReflectometry.interface import InterfaceFactory as Interface
from EasyReflectometry.fitting import Fitter
from EasyReflectometry.plot import plot

One of benefits of using a Jupyter Notebook for our analysis is improved reproducibility, to ensure this, below we share the version of the software packages being used.

In [ ]:
print(f'EasyReflectometry: {EasyReflectometry.__version__}')
print(f'refnx: {refnx.__version__}')

## Reading in experimental data

`EasyReflectometry` has support for the `.ort` file format, a [standard file format for reduced reflectivity data developed by the Open Reflectometry Standards Organisation](https://www.reflectometry.org/working_groups/file_formats/). 
To load in a dataset, we use the `load` function. 

In [ ]:
data = load('_static/example.ort')

The function about will load the file into a [*scipp* Dataset](https://scipp.github.io/user-guide/data-structures.html#Dataset) object. 
This offers some nice visualisations of the data, including the HTML view. 

In [ ]:
data

`EasyReflectometry` also includes a custom plotting function for the data. 

In [ ]:
plot(data)

## Building our model

Now that we have read in the experimental data that we want to analyse, it is necessary that we construct some model that describes what we **think** the system looks like. 
The construction of this models is discussed in detail in the [model-dependent analysis](https://www.reflectometry.org/isis_school/2_model_dependent_analysis/what_is_model_dependent_analysis.html) and [reflectometry slab models](https://www.reflectometry.org/isis_school/3_reflectometry_slab_models/the_slab_model.html) sections of the ISIS Virtual Reflectometry Training Course on [neutron reflectometry fitting](https://www.reflectometry.org/isis_school/intro.html). 

The system that we are investigating consists of four layers (with the top and bottom as semi-finite super- and sub-phases). 
The super-phase (where the neutrons are incident first) is a silicon (Si) wafer and as a process of the sample preparation there is anticipated to by a layer of silicon dioxide (SiO<sub>2</sub>) on this material.
Then a polymer film has been attached to the silicon dioxide by some chemical method and this polymer film is solvated in a heavy water (D<sub>2</sub>O) which also makes up the sub-phase of the system. 
This is shown pictorially below, as a slab model. 

<center>
    <img src='_static/polymer_film.png' alt='A slab model description of the polymer film system.' width='300px'></img>
</center>
<center>
    A slab model description of the polymer film system (note that the layers are <b>not</b> to scale), showing the four layers of silicon, silicon dioxide, the polymer film and the heavy water subphase.
</center>

In order to constuct this model in `EasyReflecotmetry`, first we must construct objects for each of the materials that will compose the layers.
These objects should be of type `Material`, when constructed `from_pars` the arguments are the real and imaginary components of the scattering length density (in units of 10<sup>-6</sup>Å<sup>-2</sup>) and some name for the material. 

In [ ]:
si = Material.from_pars(2.07, 0, 'Si')
sio2 = Material.from_pars(3.47, 0, 'SiO2')
film = Material.from_pars(2.0, 0, 'Film')
d2o = Material.from_pars(6.36, 0, 'D2O')

We can investigate the properties of one of these objects as follows.

In [ ]:
film

Next we will produce layers from each of these materials, of type `Layer`. The `from_pars` constructor for these take the material, a thickness and a interfacial roughness (on the top of the layer). The thickness and roughness values are both in Å. 

In [ ]:
si_layer = Layer.from_pars(si, 0, 0, 'Si layer')
sio2_layer = Layer.from_pars(sio2, 30, 3, 'SiO2 layer')
film_layer = Layer.from_pars(film, 250, 3, 'Film Layer')
superphase = Layer.from_pars(d2o, 0, 3, 'D2O Subphase')

Again, we can probe the properties of the layer as such.

In [ ]:
film_layer

Given that the silicon and silicon dioxide layer both compose the solid subphase, it can be helpful to combine these as a `Multilayer` [item type](./item_library.html#multilayer) in our code. 

In [ ]:
subphase = MultiLayer.from_pars([si_layer, sio2_layer], name='Si/SiO2 Superphase')

These objects are then combined as a `Structure`, where the constructor takes a series of layers (or some more complex `EasyReflectometry` [item](./item_library.html)) and, optionally, some name for the structure.

In [ ]:
structure = Structure.from_pars(subphase, film_layer, superphase, name='Film Structure')

This structure can be investigated from the string representation like the other objects. 

In [ ]:
structure

## Constructing the model

The structure of the system under investigation is just part of the analysis story. 
It is also necessary to describe the instrumental parameters, namely the background level, the resolution and some option to scale the data in the *y*-axis. 
<div class="alert alert-info">
    
Note
    
Currently, only constant with resolution is supported. We are working to include more complex resolution in future.

</div>

the `Model` constructor takes our structure, a scale factor, a uniform background level and a resolution width. 

In [ ]:
model = Model.from_pars(structure, 1, 1e-6, 0.02, 'Film Model')

From this object, we can investigate all of the parameters of our model.

In [ ]:
model

## Setting varying parameters

Now that the model is fully constructed, we can select the parameters in our model that should be varied. 
Below we set the thickness of the SiO<sub>2</sub> and film layers to vary along with the real scattering length density of the film and all of the roughnesses. 

In [ ]:
# Thicknesses
sio2_layer.thickness.bounds = (15, 50)
film_layer.thickness.bounds = (200, 300)
# Roughnesses
sio2_layer.roughness.bounds = (1, 15)
film_layer.roughness.bounds = (1, 15)
superphase.roughness.bounds = (1, 15)
# Scattering length density
film_layer.material.sld.bounds = (0.1, 3)

In addition to these variables of the structure, we will also vary the background level and scale factor. 

In [ ]:
# Background
model.background.bounds = (1e-8, 1e-5)
# Scale
model.scale.bounds = (0.5, 1.5)

## Choosing our calculation engine

The `EasyReflectometry` package enables the calculation of the reflectometry profile using either [*refnx*](https://refnx.readthedocs.io/) or [*Refl1D*](https://refl1d.readthedocs.io/en/latest/).
For this tutorial, we will stick to the current default, which is *refnx*. 
The calculator must be created and associated with the model that we are to fit. 

In [ ]:
interface = Interface()
model.interface = interface

We can check the calculation engine currently in use as follows. 

In [ ]:
print(interface.current_interface.name)

## Performing an optimisation

The optimisation of our model is achieved with a `Fitter`, which takes our model and calculator. 

In [ ]:
fitter = Fitter([model], [interface.fit_func])

To actually perform the optimisation, we must pass our `data` object created from the experimental data. 
This will return a new `sc.Dataset` with the result of out analysis, and the model will be updated in place. 

In [ ]:
analysed = fitter.fit(data)

In [ ]:
analysed

The same `plot` function that was used on the raw data can be used for this `analysed` object and will show the best fit simulated data and the associated scattering length density profile. 

In [ ]:
plot(analysed)

Finally, from the string representation of the parameters we can obtain information about the optimised values.

In [ ]:
model

We note here that the results obtained are very similar to those from the [*refnx* tutorial](https://refnx.readthedocs.io/en/latest/getting_started.html#Fitting-a-neutron-reflectometry-dataset), which is hardly surprising given that we have used the *refnx* engine in this example.